# Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux était utilisée pour de la nourriture ? 

In [222]:
import pandas as pd

In [223]:
#Pour déterminé le besoin nutritionnel moyen j'ai fais la moyenne de 32 données tirées des sources communiquées en bas de page.
besoin_nutritionnel_moyen_kcal = 2222 * 365
besoin_nutritionnel_moyen_prot = round((69.192 * 365), 2)

In [224]:
#Récupération des données utiles
data_population = pd.read_csv('fr_population.csv')
data_vegetaux = pd.read_csv('fr_vegetaux.csv')

#On gère nos noms de colonnes via des variables pour faciliter la maintenance du code
code_zone = "Code zone"
zone = 'Zone'
valeur = 'Valeur'
element = "Élément"
code_domaine = 'Code Domaine'
domaine = 'Domaine'
code_element = 'Code Élément'
code_produit = 'Code Produit'
produit = 'Produit'
code_annee = 'Code année'
annee = 'Année'
symbole = 'Symbole'
description_symbole = 'Description du Symbole'
unite = 'Unité'
energie_poids = 'Ratio énergie/poids en Kcal/Kg'
valeur_kg = 'Valeur en kg'
energie_poids_dispo_alim = 'Ratio énergie/poids pour la disponibilité alimentaire en kcal/kg '
proteines_poids = 'Ratio proteines/poids en g/Kg'
proteines_poids_dispo_alim = 'Ratio proteines/poids pour la disponibilité alimentaire en g/kg '

#On retire les données qui nous sont inutiles
mask0 = data_population[code_zone] != 351
mask1 = data_vegetaux[code_zone] != 351
mask2 = data_vegetaux[element] == 'Disponibilité de protéines en quantité (g/personne/jour)'
mask3 = data_vegetaux[element] == 'Aliments pour animaux'
mask4 = data_vegetaux[element] == 'Pertes'
mask5 = data_vegetaux[element] == 'Disponibilité alimentaire (Kcal/personne/jour)'
mask6 = data_vegetaux[element] == 'Nourriture'

data_population = data_population[mask0]
data_vegetaux = data_vegetaux[mask1 & (mask2 | mask3 | mask4 | mask5 | mask6)]
data_vegetaux = data_vegetaux.drop([code_zone, code_element, code_domaine, domaine, code_produit, code_annee, annee, symbole, description_symbole], axis = 1)
data_vegetaux.head()

,Zone,Élément,Produit,Unité,Valeur
5,Afghanistan,Pertes,Blé,Milliers de tonnes,775.00
6,Afghanistan,Nourriture,Blé,Milliers de tonnes,4895.00
8,Afghanistan,Disponibilité alimentaire (Kcal/personne/jour),Blé,Kcal/personne/jour,1369.00
9,Afghanistan,Disponibilité de protéines en quantité (g/pers...,Blé,g/personne/jour,36.91
15,Afghanistan,Pertes,Riz (Eq Blanchi),Milliers de tonnes,24.00


In [225]:
#Fonction qui récupère la population d'un pays donné
def recup_pop(pays) :
    pop = data_population[data_population[zone] == pays][valeur].values
    pop = int(pop * 1000)
    return pop

In [226]:
#fonction qui retourne la disponibilité alimentaire végétale mondiale par produit composée de :la disponibilité alimentaire, la nourriture destinée aux animaux et les pertes en kg
def calculer_dispo_alim_vegetaux(data) :
    
    mask1 = data[element] != 'Disponibilité alimentaire (Kcal/personne/jour)'
    mask2 = data[element] != 'Disponibilité de protéines en quantité (g/personne/jour)'
    mask3 = data[valeur] != 0
    data.loc[ : ,valeur] *= 1000000
    #On isole les lignes à traiter
    data = data[mask1 & mask2 & mask3]
    dispo_alim_mondiale = data.groupby(produit).sum(valeur).reset_index()
    return dispo_alim_mondiale

In [227]:
#Fonction qui permet de calculer le ratio energie/poids d'un produit dans chaque pays
def calculer_energie_poids(data) :
    data_ratio_values = []
    mask1 = data[element] == 'Nourriture'
    mask3 = data[element] == 'Disponibilité alimentaire (Kcal/personne/jour)'
    data = data[mask1 | mask3]
    kg = 0
    kcal = 0
    for z, p, u, v in zip(data[zone], data[produit], data[unite], data[valeur]) :
        div = False
        if u == 'Milliers de tonnes' :
            kg = v * 1000000
        else :
            kcal = v
            div = True
        if div == True :
            if kg == 0 :
                ratio = 0
            else :
                pop = recup_pop(z)
                kcal *= pop * 365
                ratio = round((kcal / kg), 2)
                ratio_values =  [z, p, ratio]
                data_ratio_values.append(ratio_values)
    data_ratio = pd.DataFrame(data_ratio_values, columns = [zone, produit, energie_poids])
    data_ratio = data_ratio[data_ratio[energie_poids] != 0]
    return data_ratio

In [228]:
#Fonction qui permet de calculer le ratio poids de proteine/poids dans chaque pays
def calculer_ratio_proteines(data) :
    data_ratio_values = []
    mask1 = data[element] == 'Nourriture'
    mask3 = data[element] == 'Disponibilité de protéines en quantité (g/personne/jour)'
    data = data[mask1 | mask3]
    kg = 0
    prot = 0
    for z, p, u, v in zip(data[zone], data[produit], data[unite], data[valeur]) :
        div = False
        if u == 'Milliers de tonnes' :
            kg = v * 1000000
        else :
            prot = v /1000
            div = True
        if div == True :
            if kg == 0 :
                ratio = 0
            else :
                pop = recup_pop(z)
                prot *= pop * 365
                ratio = round(((prot / kg) * 100), 2)
                ratio_values =  [z, p, ratio]
                data_ratio_values.append(ratio_values)
    data_ratio = pd.DataFrame(data_ratio_values, columns = [zone, produit, proteines_poids])
    data_ratio = data_ratio[data_ratio[proteines_poids] != 0]
    return data_ratio

In [229]:
#Fonction qui calcul le ratio kcal/disponibilité alimentaire végétale mondiale par produit composée de :la disponibilité alimentaire, la nourriture destinée aux animaux et les pertes en kg
def calculer_ratio_kcal_dispo_alim(data1, data2) :
    #On compare nos dataframe et on supprime les produits dont des données manquent 
    data_1_len = len(data1)
    data_2_len = len(data2)
    col1 = data1.columns.values
    col2 = data2.columns.values
    kcal_dispo_alim = []
    #On retire les lignes non renseignées dans les 2 dataframe
    if data_1_len > data_2_len :
        for p in data1[produit] :
            data3 = data2[data2[produit] == p]
            z = len(data3)
            if z == 0 :
                data1 = data1[data1[produit] != p]
            else :
                prod = data1[data1[produit] == p]
                kcal_kg = prod[col1[1]].values * float (data3[col2[1]].values)
                ratio_values = [p, round(kcal_kg, 2)]
                kcal_dispo_alim.append(ratio_values)
    else :
        for p in data2[produit] :
            data3 = data1[data1[produit] == p]
            z = len(data3)
            if z == 0 :
                data2 = data2[data2[produit] != p]
            else :
                prod = data2[data2[produit] == p]
                kcal_kg = float (prod[col2[1]].values) * float (data3[col1[1]].values)
                ratio_values = [p, round(kcal_kg, 2)]
                kcal_dispo_alim.append(ratio_values)
    data1 = pd.DataFrame(kcal_dispo_alim, columns = [produit, energie_poids_dispo_alim]).set_index(produit)
    donnee = data1.sum()
    donnee = donnee[0]
    return donnee

In [230]:
#Fonction qui calcul le ratio proteine/disponibilité végétale mondiale par produit composée de :la disponibilité alimentaire, la nourriture destinée aux animaux et les pertes en kg
def calculer_ratio_prot_dispo_alim(data1, data2) :
    #On compare nos dataframe et on supprime les produits dont des données manquent 
    data_1_len = len(data1)
    data_2_len = len(data2)
    col1 = data1.columns.values
    col2 = data2.columns.values
    prot_dispo_alim = []
    #On retire les lignes non renseignées dans l'un des 2 dataframes
    if data_1_len > data_2_len :
        for p in data1[produit] :
            data3 = data2[data2[produit] == p]
            z = len(data3)
            if z == 0 :
                data1 = data1[data1[produit] != p]
            else :
                prod = data1[data1[produit] == p]
                prot_kg = prod[col1[1]].values * float (data3[col2[1]].values)
                ratio_values = [p, round(prot_kg, 2)]
                prot_dispo_alim.append(ratio_values)
    else :
        for p in data2[produit] :
            data3 = data1[data1[produit] == p]
            z = len(data3)
            if z == 0 :
                data2 = data2[data2[produit] != p]
            else :
                prod = data2[data2[produit] == p]
                prot_kg = float (prod[col2[1]].values) * float (data3[col1[1]].values)
                ratio_values = [p, round(prot_kg, 2)]
                prot_dispo_alim.append(ratio_values)
    data1 = pd.DataFrame(prot_dispo_alim, columns = [produit, proteines_poids_dispo_alim]).set_index(produit)
    donnee = data1.sum()
    donnee = donnee[0]
    return donnee

In [231]:
population_mondiale = data_population[valeur].sum() * 1000

In [232]:
data_ratio_kcal = calculer_energie_poids(data_vegetaux)
data_ratio_kcal_total = data_ratio_kcal.groupby(produit).mean(valeur).reset_index()
data_ratio_kcal_total.head()

,Produit,Ratio énergie/poids en Kcal/Kg
0,"Agrumes, Autres",248.981351
1,Aliments pour enfants,3690.832048
2,Ananas,439.413223
3,Arachides Decortiquees,5702.900368
4,Avoine,2143.296145


In [233]:
data_ratio_proteines = calculer_ratio_proteines(data_vegetaux)
data_ratio_proteines_total = data_ratio_proteines.groupby(produit).mean(valeur).reset_index()
data_ratio_proteines_total.head()

,Produit,Ratio proteines/poids en g/Kg
0,"Agrumes, Autres",0.443191
1,Aliments pour enfants,14.883191
2,Ananas,0.307431
3,Arachides Decortiquees,25.146812
4,Avoine,8.446593


In [234]:
dispo_alim_vegetaux = calculer_dispo_alim_vegetaux(data_vegetaux)
dispo_alim_vegetaux.head()

,Produit,Valeur
0,"Agrumes, Autres",1.216900e+10
1,Aliments pour enfants,7.620000e+08
2,Ananas,2.350200e+10
3,Arachides Decortiquees,1.351000e+10
4,Avoine,2.066800e+10


In [235]:
#Disponibilité alimentaire mondiale en énergie kcal
ratio_kcal_dispo_alim_mondial = calculer_ratio_kcal_dispo_alim(data_ratio_kcal_total, dispo_alim_vegetaux)

In [236]:
#Disponibilité alimentaire mondiale en proteine g/kg
ratio_proteines_dispo_alim_mondial = calculer_ratio_prot_dispo_alim(data_ratio_proteines_total, dispo_alim_vegetaux)

In [237]:
#Nombre d'humain qui pourraient être nourris avec la disponibilité intérieure en énergie
humains_nourri_dispo_alim_kcal = int(ratio_kcal_dispo_alim_mondial / besoin_nutritionnel_moyen_kcal)
humains_nourri_dispo_alim_kcal_pourcent = round((humains_nourri_dispo_alim_kcal / population_mondiale) * 100, 2)

In [238]:
#Nombre d'humain qui pourraient être nourris avec la disponibilité intérieure en proteines
humains_nourri_dispo_alim_proteines = int (ratio_proteines_dispo_alim_mondial / besoin_nutritionnel_moyen_prot)
humains_nourri_dispo_alim_proteines_pourcent = round((humains_nourri_dispo_alim_proteines / population_mondiale) * 100, 2)

In [239]:
#Réponses question 8 :
print("Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux, la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ?")
print(" - D'un point de vue énergétique : ", humains_nourri_dispo_alim_kcal, " soit : ", humains_nourri_dispo_alim_kcal_pourcent, "%")
print(" - D'un point de vue protéines : ", humains_nourri_dispo_alim_proteines, " soit : ", humains_nourri_dispo_alim_proteines_pourcent, "%")

Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux, la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ?
 - D'un point de vue énergétique :  11716569185  soit :  167.44 %
 - D'un point de vue protéines :  846863419  soit :  12.1 %
